In [10]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial

from src.trainer.trainer_classifier import Trainer_classifier
from src.trainer.trainer_VAE import Trainer_VAE
from src.trainer.model_class import Model_class

from src.models.classifiers import *
from src.models.autoencoders import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
df = pd.read_csv('../data/df_to_enc.csv')
df = df.drop(columns = ['Machine failure'])

In [12]:
HIDDEN_PARAM = 512
LATENT_REPR = 4
BATCH_SIZE = 1024

In [13]:
X_train, X_test = train_test_split(df, shuffle=True, random_state=42)

In [14]:
train_dataset = EncoderDataset(X_train)
val_dataset = EncoderDataset(X_test)

In [15]:
train_dl = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [16]:
classifier = Baseline_classifier(50, HIDDEN_PARAM)

In [17]:
autoencoder = Autoencoder(classifier.classifier, 5)

In [20]:
loss = Encoder_loss(nn.MSELoss())
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=autoencoder,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=20)

wandb_init_params = dict(
    name=f'Autoenc_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [21]:
trainer = Trainer_classifier(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/',
                  saving_model=False
                  )

In [22]:
trainer.train_model(learning_params)
wandb.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█▇▇▆▆▅▅▅▄▄▃▃▃▃▂▂▂▁▁▁
test_loss,▃▁▃▂▄▇▆█▇█▆▇▆▄▄▃▅▄▄▄
train_loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
lr,0.00358
test_loss,1.00819
train_loss,0.2012


In [23]:
vae = VAE(classifier.classifier, 5)

In [24]:
loss = vae_loss(vae_loss_function)
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=vae,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.95)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=20)

wandb_init_params = dict(
    name=f'VAE_HidParam-{HIDDEN_PARAM}_Latent-{LATENT_REPR}',
    project="Internship_project",
    dir = '../logs/'
)

In [25]:
trainer = Trainer_VAE(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/autoencoder/',
                  saving_model=False
                  )

In [26]:
trainer.train_model(learning_params)
wandb.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█▇▇▆▆▅▅▅▄▄▃▃▃▃▂▂▂▁▁▁
test_loss,██▄▅▂▄▂▃▂▃▂▂▃▁▂▁▃▂▂▃
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
lr,0.00358
test_loss,1.05438
train_loss,1.01231
